<h1> Gradient Enhanced Effective Quadratures I

In this two part tutorial, we demonstrate how we can construct polynomial approximants using effectively subsampled quadratures with gradient evaluations of a model. 

In [1]:
%matplotlib inline
from effective_quadratures.parameter import Parameter
from effective_quadratures.polynomial import Polynomial
from effective_quadratures.indexset import IndexSet
from effective_quadratures.computestats import Statistics
from effective_quadratures.effectivequads import EffectiveSubsampling
from effective_quadratures.utils import meshgrid, evalfunction, evalgradients
import numpy as np
import matplotlib.pyplot as plt
import math
np.set_printoptions(precision=3)

Consider the following function and its gradient:
$$f(x_1, x_2) = exp( x_1 + x_2),$$
$$\nabla f=\left(\begin{array}{c}
exp( x_1 + x_2) + \gamma\\
exp( x_1 + x_2) + \gamma
\end{array}\right).$$

We start by defining our computational model:

In [2]:
def fun(x):
    return np.exp(x[0] + x[1])

def fungrad(x):
    return [np.exp(x[0] + x[1]), np.exp(x[0] + x[1])] 

def fungradnoised(x):
    return [np.exp(x[0] + x[1]) - 0.001*np.random.rand(1), np.exp(x[0] + x[1])- 0.001*np.random.rand(1)] 

In [3]:
value = 11
x1 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
x2 = Parameter(param_type="Uniform", lower=-1, upper=1, points=value, derivative_flag=1)
parameters = [x1, x2]
hyperbolic_cross = IndexSet("Hyperbolic basis", orders=[value-1,value-1], q=1.0)

# Without gradients!
esq = EffectiveSubsampling(parameters, hyperbolic_cross)
esq.set_no_of_evals(esq.no_of_basis_terms) # in effect making it a square system!
x = esq.computeCoefficients(fun, 'weighted')

# Now with gradients
print esq.no_of_basis_terms
esq.set_no_of_evals(20)
x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
print x1
x1b =  esq.computeCoefficients(fun, fungrad,'equality')
weighted_error = np.linalg.norm(x-x1, 2)
equality_error = np.linalg.norm(x-x1b, 2)

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
print weighted_error, equality_error

The total number of basis terms in this basis is:

In [ ]:
#esq.set_no_of_evals(evals)
#cols_to_prune = 5
#esq.prune(cols_to_prune)
#x1 =  esq.computeCoefficients(fun, fungrad,'weighted')
#x1b =  esq.computeCoefficients(fun, fungrad,'equality')
#xnew =  x[0:(terms-cols_to_prune)]
#weighted_error = np.linalg.norm(xnew-x1, 2)
#equality_error = np.linalg.norm(xnew-x1b, 2)
#print weighted_error, equality_error